In [79]:
import pandas as pd
from scipy.stats import percentileofscore

# Load ILB data
df = pd.read_csv('/Users/jakecardonick/Desktop/ILBStats2024.csv')

# Filter for ILBs with enough snaps
df = df[(df['position'] == 'LB') & (df['snap_counts_defense'] >= 200)].copy()

# Convert all relevant columns to numeric
cols = [
    'assists', 'batted_passes', 'catch_rate', 'forced_fumbles', 'fumble_recoveries',
    'hurries', 'interceptions', 'missed_tackle_rate', 'qb_rating_against',
    'penalties', 'sacks', 'tackles', 'tackles_for_loss', 'total_pressures',
    'snap_counts_defense', 'player_game_count'
]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Per-snap metrics
df['assists_ps'] = df['assists'] / df['snap_counts_defense']
df['batted_passes_ps'] = df['batted_passes'] / df['snap_counts_defense']
df['forced_fumbles_ps'] = df['forced_fumbles'] / df['snap_counts_defense']
df['fumble_recoveries_ps'] = df['fumble_recoveries'] / df['snap_counts_defense']
df['hurries_ps'] = df['hurries'] / df['snap_counts_defense']
df['interceptions_ps'] = df['interceptions'] / df['snap_counts_defense']
df['penalties_ps'] = df['penalties'] / df['snap_counts_defense']
df['sacks_ps'] = df['sacks'] / df['snap_counts_defense']
df['tackles_ps'] = df['tackles'] / df['snap_counts_defense']
df['tfl_ps'] = df['tackles_for_loss'] / df['snap_counts_defense']
df['pressures_ps'] = df['total_pressures'] / df['snap_counts_defense']

# New: snaps per game
df['snaps_per_game'] = df['snap_counts_defense'] / df['player_game_count']

# Positive and negative metrics
positive = [
    'assists_ps', 'batted_passes_ps', 'forced_fumbles_ps', 'fumble_recoveries_ps',
    'hurries_ps', 'interceptions_ps', 'sacks_ps', 'tackles_ps', 'tfl_ps',
    'pressures_ps', 'snaps_per_game'
]
negative = ['catch_rate', 'missed_tackle_rate', 'qb_rating_against', 'penalties_ps']

# Percentile conversion
for col in positive:
    df[f'{col}_pct'] = df[col].apply(lambda x: percentileofscore(df[col].dropna(), x))
for col in negative:
    df[f'{col}_pct'] = df[col].apply(lambda x: 100 - percentileofscore(df[col].dropna(), x))

# --- Editable weight section ---
weights = {
    'batted_passes_ps_pct': 14,
    'forced_fumbles_ps_pct': 32,
    'fumble_recoveries_ps_pct': 18,
    'hurries_ps_pct': 14,
    'interceptions_ps_pct': 24,
    'sacks_ps_pct': 10,
    'tackles_ps_pct': 10,
    'tfl_ps_pct': 16,
    'pressures_ps_pct': 14,
    'catch_rate_pct': 8,
    'qb_rating_against_pct': 18,
    'snaps_per_game_pct' : 36, 
    'missed_tackle_rate_pct' : 8,
    'assists_ps_pct' : 4
}
# -------------------------------

# Score with custom weights
numerator = sum(df[col] * w for col, w in weights.items())
denominator = sum(weights.values())
df['ILBScore'] = numerator / denominator

# Export
export_cols = ['player', 'ILBScore'] + list(weights.keys())
df_sorted = df.sort_values('ILBScore', ascending=False)
df_sorted[export_cols].to_csv('/Users/jakecardonick/Desktop/ILBScoreResults2024.csv', index=False)

# Show top 50
print(df_sorted[export_cols].head(50).to_string(index=False))


                 player  ILBScore  batted_passes_ps_pct  forced_fumbles_ps_pct  fumble_recoveries_ps_pct  hurries_ps_pct  interceptions_ps_pct  sacks_ps_pct  tackles_ps_pct  tfl_ps_pct  pressures_ps_pct  catch_rate_pct  qb_rating_against_pct  snaps_per_game_pct  missed_tackle_rate_pct  assists_ps_pct
              Zack Baun 82.048673                  70.0                   97.0                      89.0            73.0                  96.0          63.5            95.0        88.0              73.0            77.0                   96.0                71.0                    54.0            46.0
Jeremiah Owusu-Koramoah 73.792035                  94.0                   84.0                      30.5            95.0                  91.0          93.0            47.0        97.0              95.0            49.5                   86.0                65.0                     2.0             7.0
        Edgerrin Cooper 72.424779                  88.0                   77.0                